In [54]:
from queue import PriorityQueue ## put and get and empty
import copy
import numpy as np
import pygame
import cv2

In [55]:
def obstacle_1(current_node_state): ## circle
    x,y=current_node_state
    f=(x-300)**2+(y-185)**2-(45)**2 ## clearance is included 40+5
    if f<=0:
        return True ## the point is on or in the obstacle
    else: 
        return False ## not in the obstacle
     

In [56]:
def obstacle_2(current_node_state): ## two triangles with clearance
    x,y=current_node_state
    f1=25*x-79*y+13715
    f2=6*x-7*y+780
    f3=16*x+5*y-2180
    f4=85*x+69*y-15825
    if (f1>=0 and f2<=0 and f4>=0) or (f3<=0 and f2>=0 and f4>=0):
        return True ## xy is in the obstacle
    else:
        return False ## xy is not in obstacle

In [57]:
def obstacle_3(current_node_state): ## hexagon with clearance
    x,y=current_node_state
    f1=0.577*x+y-261.677
    f2=x-240
    f3=0.577*x-y-61.628
    f4=0.577*x+y-169.252
    f5=x-160
    f6=0.577*x-y+30.698
    if (f1<=0 and f2<=0 and f3<=0 and f4>=0 and f5>=0 and f6>=0):
        return True ## xy is in obstacle
    else:
        return False ##xy is not in obstacle

In [58]:
def Actions(current_node): ## input is node ## output is a list nodes in tuple(cost_to_come, parent, (i,j))
    i,j=current_node[2]
    cost=current_node[0]
    parent=current_node[3]
    action_set=[]
    if 0<=i-1<=ilen and 0<=j<=jlen: ## top action
        c2c=np.round(cost+1,4)
        action_set.append((c2c,parent,(i-1,j)))
    
    if 0<=i+1<=ilen and 0<=j+1<=jlen: ## top right
        c2c=np.round(cost+1.4,4)
        action_set.append((c2c,parent,(i+1,j+1)))
    
    
    if 0<=i<=ilen and 0<=j+1<=jlen: ## right action
        c2c=np.round(cost+1,4)
        action_set.append((c2c,parent,(i,j+1)))
        
    if 0<=i-1<=ilen and 0<=j+1<=jlen: ## bottom right
        c2c=np.round(cost+1.4,4)
        action_set.append((c2c,parent,(i-1,j+1)))
    
    if 0<=i+1<=ilen and 0<=j<=jlen: ## bottom action
        c2c=np.round(cost+1,4)
        action_set.append((c2c,parent,(i+1,j)))
    
    if 0<=i-1<=ilen and 0<=j-1<=jlen: ## bottom left
        c2c=np.round(cost+1.40,4)
        action_set.append((c2c,parent,(i-1,j-1)))
        
    if 0<=i<=ilen and 0<=j-1<=jlen: ## left action
        c2c=np.round(cost+1,4)
        action_set.append((c2c,parent,(i,j-1)))
    
    
    if 0<=i+1<=ilen and 0<=j-1<=jlen: ## top left
        c2c=np.round(cost+1.4,4)
        action_set.append((c2c,parent,(i+1,j-1)))
    
    
        
        
        
    return action_set ## list of nodes in tuple(cost_to_come, parent, (i,j))

In [59]:
def check_closed_list(current_node_state): ## check is state is in closed list
    for i in closed_list:
        if current_node_state==i[2]:
            return True ## it is in closed list
    else:
        return False ## not in closed list

In [60]:
def backtrack(current_node): ## backtrack for optimal path
    parent=current_node[1]
    index=current_node[3]
    path=[]
    path.append([current_node])
    while not index==0:
        path.append([node for node in closed_list if node[3]==parent])
        parent=path[-1][-1][1]
        index=path[-1][-1][0]
    final=[path[:][:][i][0][2] for i in range(len(path))]
    return final ## list of all states covered for optimal path

In [51]:
start_state_x=int(input("Type the start position x value:  " ))
start_state_y=int(input("Type the start position y value:  "))
goal_state_x=int(input('Type the goal  position x value: '))
goal_state_y=int(input('Type the goal  position y value: ' ))
ilen=240 ## map x axis length
jlen=390 ## map y axis length
open_list= PriorityQueue()
closed_list=[]
index=0
visited_list=[]


In [61]:
start_state=(start_state_x,start_state_y)
goal_state=(goal_state_x,goal_state_y)

In [65]:
##check if start state and goal state are not in obstacle space
if obstacle_1(start_state) or obstacle_2(start_state) or obstacle_3(start_state): 
    print("The start state is in obstacle space")

if obstacle_1(goal_state) or obstacle_2(goal_state) or obstacle_3(goal_state):
    print("The goal state is in obstacle space")

In [67]:
start_node=(0,"NA",start_state, index) ## make a start node
open_list.put(start_node) ## push start node in open list


In [68]:
index=0
while not open_list.empty():
    current_node=open_list.get() ## pop out of open list
    closed_list.append(current_node) ## add to closed list
    visited_list.append(current_node) ## add to visited list which will be used later for visualization
    action_set=Actions(current_node) ## find possible actions list
    print(current_node,"is current node")
    if current_node[2]==goal_state: ## check for goal state
        print("goal node found")
        path=backtrack(current_node) ## backtracking if goal has reached

        break;
    
    for node in action_set: ## for every node in action
        ## check if node is in obstacle space
        if obstacle_1(node[2]) or obstacle_2(node[2]) or obstacle_3(node[2]): 
            #print(node, " is in obstacle space")
            continue ## do not do the rest of instructions and check for next node
        
        ## check if node is in closed list
        if check_closed_list(node[2]):
            #print(node, "is in closed list")
            continue ## do not do the rest of instructions and check for next node
        
        ##check if node is in open list update or push into open list
        ##print(index)
        for item in list(open_list.queue):
            if node[2]==item[2]:
            #print(node[2], " is in open list")
            ## check if the cost is lesser and update
                if node[0]<item[0]: ## comparing the cost to come
                    item=(node[0],node[1],node[3]) ## update the cost to come
                    #item[1]==node[1] ## update the parent
                    print(item, "node updated")
                break;
                
        
        else:
            #print(node, "node not in open list")
            index=index+1
            node=node+(index,) ## add index to each valid node
            open_list.put(node)   ## add to open list
        
        
        
        
    
        
        
        
    

(0, 'NA', (0, 0), 0) is current node
(1, 0, (0, 1), 2) is current node
(1, 0, (1, 0), 3) is current node
(1.4, 0, (1, 1), 1) is current node
(2, 2, (0, 2), 5) is current node
(2, 3, (2, 0), 7) is current node
(2.4, 2, (1, 2), 4) is current node
(2.4, 3, (2, 1), 6) is current node
(2.8, 1, (2, 2), 8) is current node
(3, 5, (0, 3), 10) is current node
(3, 7, (3, 0), 12) is current node
(3.4, 5, (1, 3), 9) is current node
(3.4, 7, (3, 1), 11) is current node
(3.8, 4, (2, 3), 13) is current node
(3.8, 6, (3, 2), 14) is current node
(4, 10, (0, 4), 17) is current node
(4, 12, (4, 0), 19) is current node
(4.2, 8, (3, 3), 15) is current node
(4.4, 10, (1, 4), 16) is current node
(4.4, 12, (4, 1), 18) is current node
(4.8, 9, (2, 4), 20) is current node
(4.8, 11, (4, 2), 21) is current node
(5, 17, (0, 5), 25) is current node
(5, 19, (5, 0), 27) is current node
(5.2, 13, (3, 4), 22) is current node
(5.2, 14, (4, 3), 23) is current node
(5.4, 17, (1, 5), 24) is current node
(5.4, 19, (5, 1), 26

In [12]:
closed_list

[(0, 'NA', (0, 0), 0),
 (1, 0, (0, 1), 2),
 (1, 0, (1, 0), 3),
 (1.4, 0, (1, 1), 1),
 (2, 2, (0, 2), 5),
 (2, 3, (2, 0), 7),
 (2.4, 2, (1, 2), 4),
 (2.4, 3, (2, 1), 6),
 (2.8, 1, (2, 2), 8),
 (3, 5, (0, 3), 10),
 (3, 7, (3, 0), 12),
 (3.4, 5, (1, 3), 9),
 (3.4, 7, (3, 1), 11),
 (3.8, 4, (2, 3), 13),
 (3.8, 6, (3, 2), 14),
 (4, 10, (0, 4), 17),
 (4, 12, (4, 0), 19),
 (4.2, 8, (3, 3), 15),
 (4.4, 10, (1, 4), 16),
 (4.4, 12, (4, 1), 18),
 (4.8, 9, (2, 4), 20),
 (4.8, 11, (4, 2), 21),
 (5, 17, (0, 5), 25),
 (5, 19, (5, 0), 27),
 (5.2, 13, (3, 4), 22),
 (5.2, 14, (4, 3), 23),
 (5.4, 17, (1, 5), 24),
 (5.4, 19, (5, 1), 26),
 (5.6, 15, (4, 4), 28),
 (5.8, 16, (2, 5), 29),
 (5.8, 18, (5, 2), 30),
 (6, 25, (0, 6), 34),
 (6, 27, (6, 0), 36),
 (6.2, 20, (3, 5), 31),
 (6.2, 21, (5, 3), 32),
 (6.4, 25, (1, 6), 33),
 (6.4, 27, (6, 1), 35),
 (6.6, 22, (4, 5), 37),
 (6.6, 23, (5, 4), 38),
 (6.8, 24, (2, 6), 39),
 (6.8, 26, (6, 2), 40),
 (7.0, 28, (5, 5), 41),
 (7, 34, (0, 7), 45),
 (7, 36, (7, 0), 47)

In [71]:
path

[(36, 190),
 (35, 189),
 (34, 188),
 (33, 187),
 (32, 186),
 (31, 185),
 (30, 184),
 (29, 183),
 (28, 182),
 (27, 181),
 (26, 180),
 (25, 179),
 (24, 178),
 (23, 177),
 (22, 176),
 (21, 175),
 (20, 174),
 (19, 173),
 (18, 172),
 (17, 171),
 (16, 170),
 (15, 169),
 (14, 168),
 (13, 167),
 (12, 166),
 (11, 165),
 (10, 164),
 (9, 163),
 (8, 162),
 (7, 161),
 (6, 160),
 (5, 159),
 (4, 158),
 (3, 157),
 (2, 156),
 (1, 155),
 (0, 154),
 (0, 153),
 (0, 152),
 (0, 151),
 (0, 150),
 (0, 149),
 (0, 148),
 (0, 147),
 (0, 146),
 (0, 145),
 (0, 144),
 (0, 143),
 (0, 142),
 (0, 141),
 (0, 140),
 (0, 139),
 (0, 138),
 (0, 137),
 (0, 136),
 (0, 135),
 (0, 134),
 (0, 133),
 (0, 132),
 (0, 131),
 (0, 130),
 (0, 129),
 (0, 128),
 (0, 127),
 (0, 126),
 (0, 125),
 (0, 124),
 (0, 123),
 (0, 122),
 (0, 121),
 (0, 120),
 (0, 119),
 (0, 118),
 (0, 117),
 (0, 116),
 (0, 115),
 (0, 114),
 (0, 113),
 (0, 112),
 (0, 111),
 (0, 110),
 (0, 109),
 (0, 108),
 (0, 107),
 (0, 106),
 (0, 105),
 (0, 104),
 (0, 103),
 (0, 

In [15]:
## compute the list of all nodes in obstacle space for visualising
all=[]
for x in range(0,400):
    for y in range(0,250):
        if obstacle_1((x,y)) or obstacle_2((x,y)) or obstacle_3((x,y)):
            all.append((x,y))

In [17]:
all

[(36, 185),
 (37, 184),
 (37, 185),
 (38, 183),
 (38, 184),
 (38, 185),
 (39, 182),
 (39, 183),
 (39, 184),
 (39, 185),
 (40, 181),
 (40, 182),
 (40, 183),
 (40, 184),
 (40, 185),
 (40, 186),
 (41, 179),
 (41, 180),
 (41, 181),
 (41, 182),
 (41, 183),
 (41, 184),
 (41, 185),
 (41, 186),
 (42, 178),
 (42, 179),
 (42, 180),
 (42, 181),
 (42, 182),
 (42, 183),
 (42, 184),
 (42, 185),
 (42, 186),
 (43, 177),
 (43, 178),
 (43, 179),
 (43, 180),
 (43, 181),
 (43, 182),
 (43, 183),
 (43, 184),
 (43, 185),
 (43, 186),
 (43, 187),
 (44, 176),
 (44, 177),
 (44, 178),
 (44, 179),
 (44, 180),
 (44, 181),
 (44, 182),
 (44, 183),
 (44, 184),
 (44, 185),
 (44, 186),
 (44, 187),
 (45, 174),
 (45, 175),
 (45, 176),
 (45, 177),
 (45, 178),
 (45, 179),
 (45, 180),
 (45, 181),
 (45, 182),
 (45, 183),
 (45, 184),
 (45, 185),
 (45, 186),
 (45, 187),
 (46, 173),
 (46, 174),
 (46, 175),
 (46, 176),
 (46, 177),
 (46, 178),
 (46, 179),
 (46, 180),
 (46, 181),
 (46, 182),
 (46, 183),
 (46, 184),
 (46, 185),
 (46

### VISUALIZATION

In [99]:
### visualise obstacle space
user_int = np.zeros((250,400,3),np.uint8)

for item in all: 
    x = item[0]
    y = item[1]
    user_int[(y,x)]=[0,255,255]  ### mark with yellow
user_int = np.flipud(user_int) 
user_int_for_backtrack = user_int.copy()
user_int_for_visited = user_int.copy()
user_int_for_visited = cv2.resize(user_int_for_visited,(1200,750))
# #obstacle map
cv2.imshow('Obstacle space',user_int)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [100]:
pygame.init()

display_width = 400
display_height = 250

gameDisplay = pygame.display.set_mode((display_width,display_height))
pygame.display.set_caption('Animation of Dijkstra')

black = (0,0,0)
white = (0,255,255)

surf = pygame.surfarray.make_surface(visit)

clock = pygame.time.Clock()
done = False
while not done:
    for event in pygame.event.get():   ## to exit pygame
        if event.type == pygame.QUIT:  
            done = True   
 
    gameDisplay.fill(black) ## black space
    for i in range(len(visited_list)): ## iterate through visited list
        pygame.time.wait(0)
        x = visited_list[i][2][0]
        y1 = visited_list[i][2][1]
        y = abs(249-y1)
        pygame.draw.rect(gameDisplay, white, [x,y,1,1])
        pygame.display.flip()
    
    for i in range(len(path)):## optimal path
        pygame.time.wait(5)
        x = path[i][0]
        y2 = path[i][1]
        y = abs(249-y2)
        pygame.draw.rect(gameDisplay, (0,0,255), [x,y,1,1])
        pygame.display.flip()
        pygame.time.wait(10)
    done = True
pygame.quit()




In [101]:


#visited path
for i in range(len(visited_list)):
    x = visited_list[i][2][0]
    y1 = visited_list[i][2][0]
    user_int_for_backtrack[(249-y1,x)]=[255,0,0] #setting every backtracked pixel to white
visit= cv2.resize(user_int_for_backtrack,(1200,750))
cv2.imshow('visited',visit)
cv2.waitKey(0)
cv2.destroyAllWindows()

#backtracked path
for i in range(len(path)):
    x = path[i][0]
    y2 = path[i][1]
    # y = abs(249-y2)
    user_int_for_backtrack[(249-y2,x)]=[0,255,0] #setting every backtracked pixel to green
#showing the final backtracked path
# user_int_for_backtrack = np.flipud(user_int_for_backtrack)
optimal = cv2.resize(user_int_for_backtrack,(1200,750))
cv2.imshow('new_backtracked',optimal)
cv2.waitKey(0)
cv2.destroyAllWindows()